# Phase 6: Binary Classification Training

## Overview
Train a binary classifier to determine if character actions are consistent with their backstory.

## Task
- Input: Model-ready text pairs (backstory + chunk)
- Output: Binary label (1 = Consistent, 0 = Contradict)
- Loss: Cross-Entropy
- Metrics: Accuracy, F1, Confusion Matrix

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import json

PROJECT_ROOT = Path("/root/DataDivas_KDSH_2026")
DATA_DIR = PROJECT_ROOT / "Data"
PHASE5_OUTPUT = PROJECT_ROOT / "phase5_output"
PHASE6_OUTPUT = PROJECT_ROOT / "phase6_output"
PHASE6_OUTPUT.mkdir(parents=True, exist_ok=True)

# Load feature data
features_df = pd.read_parquet(DATA_DIR / "feature_data.parquet")
print(f"Loaded {len(features_df)} samples for training")
print(f"Label distribution:\n{features_df['label'].value_counts()}")

# Load model config (prefer phase5_output)
model_selection_path = PHASE5_OUTPUT / "model_selection.json"
if not model_selection_path.exists():
    model_selection_path = DATA_DIR / "model_selection.json"

with open(model_selection_path) as f:
    model_config = json.load(f)

model_name = model_config['primary_model']['huggingface_name']
print(f"Using model: {model_name}")

In [ ]:
# Prepare entry-level training data
entry_data = features_df.groupby('entry_id').agg({
    'model_input': 'first',
    'backstory': 'first',
    'chunk_text': lambda x: ' '.join(x),
    'label': 'first',
    'character_name': 'first',
    'violence_count': 'sum',
    'helping_count': 'sum',
    'deception_count': 'sum',
    'fear_count': 'sum',
    'anger_count': 'sum',
    'love_count': 'sum',
    'good_deed_count': 'sum',
    'bad_deed_count': 'sum',
}).reset_index()

print(f"Prepared {len(entry_data)} training samples")

# Train/val split
train_df, val_df = train_test_split(
    entry_data, test_size=0.2, stratify=entry_data['label'], random_state=42
)
print(f"Train: {len(train_df)}, Val: {len(val_df)}")

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW

class ConsistencyDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.data = dataframe.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        encoding = self.tokenizer(
            row['model_input'],
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'label': torch.tensor(row['label'], dtype=torch.long)
        }

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

print(f"Tokenizer: {model_name}")
print(f"Model params: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
def train_epoch(model, dataloader, optimizer, device):
    model.train()
    total_loss = 0
    predictions, labels = [], []
    
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        batch_labels = batch['label'].to(device)
        
        optimizer.zero_grad()
        logits = model(input_ids=input_ids, attention_mask=attention_mask).logits
        loss = torch.nn.functional.cross_entropy(logits, batch_labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        predictions.extend(preds)
        labels.extend(batch_labels.cpu().numpy())
    
    return {
        'loss': total_loss / len(dataloader),
        'accuracy': accuracy_score(labels, predictions),
        'f1': f1_score(labels, predictions, average='weighted')
    }

def evaluate(model, dataloader, device):
    model.eval()
    total_loss = 0
    predictions, labels = [], []
    
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            batch_labels = batch['label'].to(device)
            
            logits = model(input_ids=input_ids, attention_mask=attention_mask).logits
            loss = torch.nn.functional.cross_entropy(logits, batch_labels)
            
            total_loss += loss.item()
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            predictions.extend(preds)
            labels.extend(batch_labels.cpu().numpy())
    
    return {
        'loss': total_loss / len(dataloader),
        'accuracy': accuracy_score(labels, predictions),
        'f1': f1_score(labels, predictions, average='weighted'),
        'predictions': predictions,
        'labels': labels,
        'confusion_matrix': confusion_matrix(labels, predictions)
    }

print("Training functions defined")

In [ ]:
# Setup training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

train_dataset = ConsistencyDataset(train_df, tokenizer)
val_dataset = ConsistencyDataset(val_df, tokenizer)

BATCH_SIZE = 16
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = model.to(device)
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)

print(f"Train batches: {len(train_loader)}, Val batches: {len(val_loader)}")

In [ ]:
# Training loop
NUM_EPOCHS = 3
best_f1 = 0
history = []

print("=" * 50)
TRAINING
=" * 50)

for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch {epoch + 1}/{NUM_EPOCHS}")
    
    train_metrics = train_epoch(model, train_loader, optimizer, device)
    val_metrics = evaluate(model, val_loader, device)
    
    history.append({
        'epoch': epoch + 1,
        'train_loss': train_metrics['loss'],
        'train_acc': train_metrics['accuracy'],
        'train_f1': train_metrics['f1'],
        'val_loss': val_metrics['loss'],
        'val_acc': val_metrics['accuracy'],
        'val_f1': val_metrics['f1']
    })
    
    print(f"  Train - Loss: {train_metrics['loss']:.4f}, Acc: {train_metrics['accuracy']:.4f}, F1: {train_metrics['f1']:.4f}")
    print(f"  Val   - Loss: {val_metrics['loss']:.4f}, Acc: {val_metrics['accuracy']:.4f}, F1: {val_metrics['f1']:.4f}")
    
    if val_metrics['f1'] > best_f1:
        best_f1 = val_metrics['f1']
        # Save best model to phase6 output
        torch.save(model.state_dict(), PHASE6_OUTPUT / "best_model.pt")
        print(f"  Saved best model (F1: {best_f1:.4f}) -> {PHASE6_OUTPUT / 'best_model.pt'}")

print("\nTraining complete")

In [ ]:
# Final results
final = evaluate(model, val_loader, device)

print("=" * 50)
FINAL RESULTS
=" * 50)
print(f"Accuracy: {final['accuracy']:.4f}")
print(f"F1 Score: {final['f1']:.4f}")
print(f"Best F1: {best_f1:.4f}")

cm = final['confusion_matrix']
print("\nConfusion Matrix:")
print(f"              Predicted")
print(f"              Contr  Cons")
print(f"Actual Contr   {cm[0][0]:4d}   {cm[0][1]:4d}")
print(f"Actual Cons    {cm[1][0]:4d}   {cm[1][1]:4d}")

print("\nClassification Report:")
print(classification_report(final['labels'], final['predictions'], 
                          target_names=['Contradict', 'Consistent']))

In [ ]:
# Save artifacts
pd.DataFrame(history).to_csv(PHASE6_OUTPUT / "training_history.csv", index=False)

results = {
    'model_name': model_name,
    'epochs': NUM_EPOCHS,
    'final_accuracy': final['accuracy'],
    'final_f1': final['f1'],
    'best_f1': best_f1,
    'train_size': len(train_df),
    'val_size': len(val_df)
}
with open(PHASE6_OUTPUT / "training_results.json", 'w') as f:
    json.dump(results, f, indent=2)

# Final report for Phase 6
import datetime
final_report = {
    'phase': 'Phase 6 - Training',
    'timestamp': datetime.datetime.utcnow().isoformat() + 'Z',
    'results': results,
    'files_saved': {
        'best_model': 'phase6_output/best_model.pt',
        'training_history': 'phase6_output/training_history.csv',
        'training_results': 'phase6_output/training_results.json'
    }
}
with open(PHASE6_OUTPUT / 'final_report.json', 'w') as f:
    json.dump(final_report, f, indent=2)

print(f"✓ Model saved: {PHASE6_OUTPUT / 'best_model.pt'}")
print(f"✓ History saved: {PHASE6_OUTPUT / 'training_history.csv'}")
print(f"✓ Results saved: {PHASE6_OUTPUT / 'training_results.json'}")
print(f"\n💾 Phase 6 outputs saved to: {PHASE6_OUTPUT}")

print("\nPhase 6 Complete! Ready for Phase 7.")